# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [2]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
  
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            

            
def InterpolateLanes(lines, imgShape, order):

    # Arrays where we will store the points(X,Y) for each lane to be fitted
    x_LeftLane = []
    y_LeftLane = []
    
    x_RightLane = []
    y_RightLane = []
    
    for line in lines:
        for x1,y1,x2,y2 in line:                       
            # Since we can't use all the points to Interpolate/Extrapolate
            # We first put together all the points (x,y) that belong to each lane looking at their slope
            if (x2-x1) != 0:
                slope = ((y2-y1)/(x2-x1))
                # left lane
                if slope < 0:                 
                    x_LeftLane.append(x1)
                    y_LeftLane.append(y1)

                    x_LeftLane.append(x2)
                    y_LeftLane.append(y2)
                else:             
                    if slope > 0:                 
                        x_RightLane.append(x1)
                        y_RightLane.append(y1)

                        x_RightLane.append(x2)
                        y_RightLane.append(y2)

    
    # Interpolate the Left Lane
    # 1) calculate polynomial (Not necesarly has to be all the time a line)
    z_LeftLane = np.polyfit(x_LeftLane, y_LeftLane, order)
    f_LeftLane = np.poly1d(z_LeftLane)
    # Where does this lane start
    x_LeftLaneStart = min(x_LeftLane)
    # Where does this lane finish
    x_LeftLaneEnd = max(x_LeftLane)
    
    
    # Interpolate the Right Lane
    # 1) calculate polynomial (Not necesarly has to be all the time a line)
    z_RightLane = np.polyfit(x_RightLane, y_RightLane, order)
    f_RightLane = np.poly1d(z_RightLane)
    # Where does this lane start
    x_RightLaneStart = min(x_RightLane)
    # Where does this lane finish
    x_RightLaneEnd = max(x_RightLane)
    
    
    return f_LeftLane, f_RightLane, x_LeftLaneStart, x_RightLaneStart, x_LeftLaneEnd, x_RightLaneEnd
    

def draw_lanes(img, lines, color=[255, 0, 0], thickness=5):
    
    f_LeftLane, f_RightLane, x_LeftLaneStart, x_RightLaneStart, x_LeftLaneEnd, x_RightLaneEnd = InterpolateLanes(lines,img.shape,2) 
    for x in range(x_LeftLaneStart,x_LeftLaneEnd,10) :
        cv2.line(img, (x, int(f_LeftLane(x))), (x+10, int(f_LeftLane(x+10))), color, thickness)
            
    for x in range(x_RightLaneStart,x_RightLaneEnd,10) :
        cv2.line(img, (x, int(f_RightLane(x))), (x+10, int(f_RightLane(x+10))), color, thickness)
        
    #print('X-Left:', XLeftLane)
    #for i in range (0, len(XLeftLane)-1):
    #    print('X1,Y1[',i,']= ', int(XLeftLane[i]),int(YLeftLane[i]))
    #    print('X2,Y2[',i+1,']= ', int(XLeftLane[i+1]),int(YLeftLane[i+1]))
    #    cv2.line(img, (int(XLeftLane[i]), int(YLeftLane[i])), (int(XLeftLane[i+1]), int(YLeftLane[i+1])), color, thickness)
            
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    #draw_lines(line_img, lines)
    draw_lanes(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def colorFilter(image, colorBoundaries):
    # loop over the boundaries
    for (lower, upper) in colorBoundaries:
        # create NumPy arrays from the boundaries
        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")

        # find the colors within the specified boundaries and apply
        # the mask
        mask = cv2.inRange(image, lower, upper)
        output = cv2.bitwise_and(image, image, mask = mask)
               
        return output

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [3]:
import os
os.listdir("test_images/")

['MyTest.jpg',
 'solidWhiteCurve.jpg',
 'solidWhiteRight.jpg',
 'solidYellowCurve.jpg',
 'solidYellowCurve2.jpg',
 'solidYellowLeft.jpg',
 'whiteCarLaneSwitch.jpg']

run your solution on all test_images and make copies into the test_images directory).

In [4]:
# --------------------------------
#         My First Test
#       Munir Jojo-Verge
# --------------------------------

def munirLaneLineDetection(image):

    # define the list of Color boundaries that we want to Filter IN (leave on the image)
    # We define a lower and higher RGB boundary
    # For this exercise we would like to leave white and yellow
    #Ref: http://www.rapidtables.com/web/color/white-color.htm
    colorBoundaries = [
       ([174, 131, 0], [255, 255, 255]) 
       #([0, 0, 0], [255, 255, 255])  
    ]    
    colorfiltered = colorFilter(image, colorBoundaries)    
        
    # Convert to grayscale
    gray = grayscale(colorfiltered)

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 3
    blur_gray = gaussian_blur(gray,kernel_size)

    # Define our parameters for Canny and apply
    low_threshold = 200 # This values for high constrast video problem
    high_threshold = 320 #low_threshold * 3
    edges = canny(blur_gray, low_threshold, high_threshold)

    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    # vertices = np.array([[(0,imshape[0]),(abs(imshape[1]/2)-10, abs(imshape[0]/2)), (abs(imshape[1]/2)+10, abs(imshape[0]/2)), (imshape[1],imshape[0])]], dtype=np.int32)
    vertices = np.array([[(0,imshape[0]),(abs(imshape[1]/2)-10, abs(imshape[0]/2)+45), (abs(imshape[1]/2)+10, abs(imshape[0]/2)+45), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges,vertices)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 #distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 30    # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 100 #minimum number of pixels making up a line
    max_line_gap =   160  # maximum gap in pixels between connectable line segment

    # Run Hough on edge detected image
    line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)

    # Draw the lines on the original image
    lines_edges = weighted_img(image, line_image, α=0.8, β=1., λ=0.)

    # Plot to find out the outcome
    #plt.imshow(lines_edges)

    return lines_edges

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [5]:
# Simple test
image = mpimg.imread('test_images/MyTest.jpg')
processedImg = munirLaneLineDetection(image)
# Plot to find out the outcome
plt.imshow(processedImg)

NameError: name 'cv2' is not defined

In [6]:
# Simple test 2 - Color Filtering
#reading in an image
image = mpimg.imread('test_images/MyTest.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
#plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

colorBoundaries = [
     ([174, 131, 0], [255, 255, 255])        
     #([0, 0, 0], [255, 255, 255]) 
    ]    
#colorBoundaries = [
#    ([17, 15, 100], [50, 56, 200]),
#    ([86, 31, 4], [220, 88, 50]),
#    ([25, 146, 190], [62, 174, 250]),
#    ([103, 86, 65], [145, 133, 128])
#    ]
colorfiltered = colorFilter(image, colorBoundaries)    
# show the images
#plt.imshow(image)
#cv2.imshow("images", np.hstack([image, colorfiltered]))
#plt.imshow(colorfiltered)

# Convert to grayscale
gray = grayscale(colorfiltered)
#plt.imshow(gray)

# Define a kernel size and apply Gaussian smoothing
kernel_size = 1
blur_gray = gaussian_blur(gray,kernel_size)
#plt.imshow(blur_gray)

# Define our parameters for Canny and apply
low_threshold = 1 # This values for high constrast video problem
high_threshold = 250 #low_threshold * 3
edges = canny(blur_gray, low_threshold, high_threshold)
#plt.imshow(edges)

# This time we are defining a four sided polygon to mask
imshape = image.shape
# vertices = np.array([[(0,imshape[0]),(abs(imshape[1]/2)-10, abs(imshape[0]/2)), (abs(imshape[1]/2)+10, abs(imshape[0]/2)), (imshape[1],imshape[0])]], dtype=np.int32)
vertices = np.array([[(0,imshape[0]),(abs(imshape[1]/2)-10, abs(imshape[0]/2)+45), (abs(imshape[1]/2)+10, abs(imshape[0]/2)+45), (imshape[1],imshape[0])]], dtype=np.int32)
masked_edges = region_of_interest(edges,vertices)
#plt.imshow(masked_edges)

# Define the Hough transform parameters
# Make a blank the same size as our image to draw on
rho = 2 #distance resolution in pixels of the Hough grid
theta = np.pi/180 # angular resolution in radians of the Hough grid
threshold = 50    # minimum number of votes (intersections in Hough grid cell)
min_line_len = 7 #minimum number of pixels making up a line
max_line_gap =   15  # maximum gap in pixels between connectable line segment

# Run Hough on edge detected image
line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)
#plt.imshow(line_image)

# Draw the lines on the original image
lines_edges = weighted_img(image, line_image, α=0.8, β=1., λ=0.)
plt.imshow(lines_edges)

This image is: <class 'numpy.ndarray'> with dimesions: (540, 960, 3)


NameError: name 'cv2' is not defined

In [7]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

ImportError: No module named 'moviepy'

In [8]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)    
    return munirLaneLineDetection(image)

Let's try the one with the solid white lane on the right first ...

In [9]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

NameError: name 'VideoFileClip' is not defined

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


#  My Reflections
The very first thing that came to my mind was "Adaptive" parameter estimation. The intuitive process to make the code work for the first and second videos (white and yellow) totally collapsed on the Challenge video. I found myself changing and tuning the entire set of parameters until it actually didn't fault. My conclusions in this iterative process were:
•	I was now clearly aware of the difficulties around this task and the safety implications around it.
•	I found out that my regression worked best if it was linear in the first 2 cases but quadratic was more effective for the challenge. I played with the degree on the regression until I over-fitted and was very good for the “Challenge” but horrible for the first 2 videos. Adaptive / Dynamic parameter selection kept coming to my mind
•	I realized how sensitive the system could be to changes on the parameters...and sensitivity is NOT a good friend of Robustness.
•	I realized that some of the mechanisms we use today to perform these tasks might not be the ideal ones. For instance, we all know that the roads are built for humans and not for computers and all the markings are made for humans to be able to "see" them. But "seeing" can be accomplished by a wide range of sensors using a very wide bandwidth sets. Basically, my point is that if we use a different type of cameras and not only play with the "visual" range of frequencies we might find that the task could be, maybe, easier...or maybe plan for the future and use a different type of lane paint or reflectors – for specific frequencies that might be robust enough to handle bad weather conditions that might scatter the emitted/reflected signal – Active vs Passive remote sensing). (all these is for other time fun discussion)
•	The parameters I left for the submission/evaluation were not the optimal ones, but the ones that did work, arguably, well enough for ALL the 3 cases, including the Challenge one without having to change any parameter. 
•	Understanding the exact meaning and impact of each parameter was challenging; especially when combined with the others that could produce a result that might not let you see the specific effects.
•	The Challenge video proved that, even when humans might find it relatively simple to navigate, the shadows, the lack of markings, the cement Vs asphalt, the contrast created by the side barrier and the road, etc...made clear the difficultness of the task and brought back, again, the idea of building an "Adaptive" approach that could adjust to all these challenges and also to the changes in light due to the time of the day or due to weather...
•	OpenCV proved to be an excellent companion to be able to solve/develop these solutions. The problem here lays in the deep knowledge on the library itself required to use and optimize the development. I need to learn more about it.
•	The “Notebook” approach is definitely rich and flexible for education purposes but it definitely makes is harder to debug.
•	In general I’m satisfied with this first assignment and I think I can learn a lot from you guys. For that I’m very thankful.

There are more things that could be added, but I beleive this is a great point to stop.

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))